In [1]:
import warnings
warnings.filterwarnings('ignore')
#from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv('../dataset/Multimodal.csv')
data

,ID,Question Number,Share Context,Share Image,Question Text,Image,Options,Answer,Explanation,QA Type,Question Type,Level of Difficulty,Knowledge Topics,General Topics,Book Label
0,0,1,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,Wakuluk most likely seeks to mitigate which of...,NaN,"{'A': 'Availability', 'B': 'Time period', 'C':...",A,Wakuluk started her career when the global mar...,Knowledge reasoning QA,text+image,Easy,"Behavioral finance, Cognitive biases, Decision...",Capital Market Expectations,Capital Market Expectations
1,1,2,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,Wakuluk’s approach to economic forecasting:,NaN,{'A': 'is flexible and limited in complexity.'...,B,Wakuluk’s approach to economic forecasting uti...,Knowledge reasoning QA,text+image,Easy,"Econometric Models, Leading Indicators, Checkl...",Capital Market Expectations,Capital Market Expectations
2,2,3,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,Wakuluk is most likely to make significant adj...,NaN,"{'A': 'Country Y only', 'B': 'Country Z only',...",B,Country Z is a developing market. Less-develop...,Knowledge reasoning QA,text+image,Easy,"Emerging Markets, Structural Changes, Market A...",Capital Market Expectations,Capital Market Expectations
3,3,4,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,Based on Exhibit 1 and Wakuluk’s assumptions a...,NaN,"{'A': 'low and bottoming.', 'B': 'approaching ...",A,Country X is predicted to be in the initial re...,Knowledge reasoning QA,text+image,Easy,"Business cycle phases, Short-term interest rat...",Capital Market Expectations,Capital Market Expectations
4,4,5,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,"Based on Exhibit 1, what capital market effect...",NaN,"{'A': 'Cyclical assets attract investors.', 'B...",B,Wakuluk’s model predicts that Country Z’s busi...,Knowledge reasoning QA,text+image,Easy,"Business cycles, Monetary policy, Yield curve",Capital Market Expectations,Capital Market Expectations
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2264,101,NaN,NaN,"There are two assets X and Y, which line is no...",images/Foundationofriskmanagement1_images/101u...,"{'A': ' A', 'B': ' B', 'C': ' C', 'D': ' D'}",D,The efficient frontier is concave above the mi...,Knowledge reasoning QA,text+image,Easy,"efficient frontier, portfolio theory",Foundation of Risk Management,foundation of risk management1
95,2266,103,NaN,NaN,Which of the following portfolios falls below ...,images/Foundationofriskmanagement1_images/103u...,"{'A': ' Portfolio A', 'B': ' Portfolio B', 'C'...",B,Portfolio B is inefficient (falls below the ef...,Knowledge reasoning QA,text+image,Easy,"Markowitz efficient frontier, portfolio theory",Foundation of Risk Management,foundation of risk management1
96,2270,107,NaN,NaN,Consider the expected returns and standard dev...,images/Foundationofriskmanagement1_images/107u...,"{'A': ' Portfolio 1', 'B': ' Portfolio 2', 'C'...",A,Portfolio 1 is not efficient because it has a ...,Knowledge reasoning QA,text+image,Easy,"mean variance efficiency, portfolio analysis",Foundation of Risk Management,foundation of risk management1
97,2695,179,NaN,NaN,A regression of a stock’s return (in percent) ...,images/QuantitativeAnalysis1_images/179u.png,{'A': ' The correlation coefficient between th...,D,\n\n$$\n\rho^2 = R^2 = \frac{92.648}{117.160} ...,Knowledge reasoning QA,text+image,Easy,"Regression, Correlation coefficient, Stock ret...",Quantitative Methods,Quantitative Analysis1


In [12]:
from openai import OpenAI
client = OpenAI(api_key="sk-VUQOgoxNjtiPxFDo895535A3635847B7A903688099089385", 
                base_url="https://api.xi-ai.cn/v1")

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding
data['QuestionTextEmbedding'] = data['Question Text'].apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
data['OptionsEmbedding'] = data['Options'].apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
data


,ID,Question Number,Share Context,Share Image,Question Text,Image,Options,Answer,Explanation,QA Type,Question Type,Level of Difficulty,Knowledge Topics,General Topics,Book Label,QuestionTextEmbeding
0,0,1,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,Wakuluk most likely seeks to mitigate which of...,NaN,"{'A': 'Availability', 'B': 'Time period', 'C':...",A,Wakuluk started her career when the global mar...,Knowledge reasoning QA,text+image,Easy,"Behavioral finance, Cognitive biases, Decision...",Capital Market Expectations,Capital Market Expectations,"[0.014189721085131168, 0.02863846719264984, 0...."
1,1,2,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,Wakuluk’s approach to economic forecasting:,NaN,{'A': 'is flexible and limited in complexity.'...,B,Wakuluk’s approach to economic forecasting uti...,Knowledge reasoning QA,text+image,Easy,"Econometric Models, Leading Indicators, Checkl...",Capital Market Expectations,Capital Market Expectations,"[-0.018121331930160522, -0.013318340294063091,..."
2,2,3,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,Wakuluk is most likely to make significant adj...,NaN,"{'A': 'Country Y only', 'B': 'Country Z only',...",B,Country Z is a developing market. Less-develop...,Knowledge reasoning QA,text+image,Easy,"Emerging Markets, Structural Changes, Market A...",Capital Market Expectations,Capital Market Expectations,"[0.032473694533109665, 0.029463937506079674, 0..."
3,3,4,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,Based on Exhibit 1 and Wakuluk’s assumptions a...,NaN,"{'A': 'low and bottoming.', 'B': 'approaching ...",A,Country X is predicted to be in the initial re...,Knowledge reasoning QA,text+image,Easy,"Business cycle phases, Short-term interest rat...",Capital Market Expectations,Capital Market Expectations,"[-0.06923175603151321, 0.0031668278388679028, ..."
4,4,5,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,"Based on Exhibit 1, what capital market effect...",NaN,"{'A': 'Cyclical assets attract investors.', 'B...",B,Wakuluk’s model predicts that Country Z’s busi...,Knowledge reasoning QA,text+image,Easy,"Business cycles, Monetary policy, Yield curve",Capital Market Expectations,Capital Market Expectations,"[0.019760508090257645, 0.036344975233078, 0.03..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2264,101,NaN,NaN,"There are two assets X and Y, which line is no...",images/Foundationofriskmanagement1_images/101u...,"{'A': ' A', 'B': ' B', 'C': ' C', 'D': ' D'}",D,The efficient frontier is concave above the mi...,Knowledge reasoning QA,text+image,Easy,"efficient frontier, portfolio theory",Foundation of Risk Management,foundation of risk management1,"[-0.011370659805834293, 0.01683899201452732, 0..."
95,2266,103,NaN,NaN,Which of the following portfolios falls below ...,images/Foundationofriskmanagement1_images/103u...,"{'A': ' Portfolio A', 'B': ' Portfolio B', 'C'...",B,Portfolio B is inefficient (falls below the ef...,Knowledge reasoning QA,text+image,Easy,"Markowitz efficient frontier, portfolio theory",Foundation of Risk Management,foundation of risk management1,"[-0.021951964125037193, -0.006178832147270441,..."
96,2270,107,NaN,NaN,Consider the expected returns and standard dev...,images/Foundationofriskmanagement1_images/107u...,"{'A': ' Portfolio 1', 'B': ' Portfolio 2', 'C'...",A,Portfolio 1 is not efficient because it has a ...,Knowledge reasoning QA,text+image,Easy,"mean variance efficiency, portfolio analysis",Foundation of Risk Management,foundation of risk management1,"[0.01953645423054695, 0.018257861956954002, 0...."
97,2695,179,NaN,NaN,A regression of a stock’s return (in percent) ...,images/QuantitativeAnalysis1_images/179u.png,{'A': ' The correlation coefficient between th...,D,\

In [13]:

data

,ID,Question Number,Share Context,Share Image,Question Text,Image,Options,Answer,Explanation,QA Type,Question Type,Level of Difficulty,Knowledge Topics,General Topics,Book Label,QuestionTextEmbeding,OptionsEmbeding
0,0,1,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,Wakuluk most likely seeks to mitigate which of...,NaN,"{'A': 'Availability', 'B': 'Time period', 'C':...",A,Wakuluk started her career when the global mar...,Knowledge reasoning QA,text+image,Easy,"Behavioral finance, Cognitive biases, Decision...",Capital Market Expectations,Capital Market Expectations,"[0.014189721085131168, 0.02863846719264984, 0....","[0.008164113387465477, 0.024835949763655663, 0..."
1,1,2,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,Wakuluk’s approach to economic forecasting:,NaN,{'A': 'is flexible and limited in complexity.'...,B,Wakuluk’s approach to economic forecasting uti...,Knowledge reasoning QA,text+image,Easy,"Econometric Models, Leading Indicators, Checkl...",Capital Market Expectations,Capital Market Expectations,"[-0.018121331930160522, -0.013318340294063091,...","[0.039293255656957626, 0.02055339515209198, 0...."
2,2,3,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,Wakuluk is most likely to make significant adj...,NaN,"{'A': 'Country Y only', 'B': 'Country Z only',...",B,Country Z is a developing market. Less-develop...,Knowledge reasoning QA,text+image,Easy,"Emerging Markets, Structural Changes, Market A...",Capital Market Expectations,Capital Market Expectations,"[0.032473694533109665, 0.029463937506079674, 0...","[-0.00390667375177145, -0.017544399946928024, ..."
3,3,4,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,Based on Exhibit 1 and Wakuluk’s assumptions a...,NaN,"{'A': 'low and bottoming.', 'B': 'approaching ...",A,Country X is predicted to be in the initial re...,Knowledge reasoning QA,text+image,Easy,"Business cycle phases, Short-term interest rat...",Capital Market Expectations,Capital Market Expectations,"[-0.06923175603151321, 0.0031668278388679028, ...","[0.019223447889089584, 0.01628788746893406, 0...."
4,4,5,Neshie Wakuluk is an investment strategist who...,['images/CapitalMarketExpectations_images/shar...,"Based on Exhibit 1, what capital market effect...",NaN,"{'A': 'Cyclical assets attract investors.', 'B...",B,Wakuluk’s model predicts that Country Z’s busi...,Knowledge reasoning QA,text+image,Easy,"Business cycles, Monetary policy, Yield curve",Capital Market Expectations,Capital Market Expectations,"[0.019760508090257645, 0.036344975233078, 0.03...","[0.02729002758860588, 0.014394286088645458, 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2264,101,NaN,NaN,"There are two assets X and Y, which line is no...",images/Foundationofriskmanagement1_images/101u...,"{'A': ' A', 'B': ' B', 'C': ' C', 'D': ' D'}",D,The efficient frontier is concave above the mi...,Knowledge reasoning QA,text+image,Easy,"efficient frontier, portfolio theory",Foundation of Risk Management,foundation of risk management1,"[-0.011370659805834293, 0.01683899201452732, 0...","[0.014560211449861526, -0.047434043139219284, ..."
95,2266,103,NaN,NaN,Which of the following portfolios falls below ...,images/Foundationofriskmanagement1_images/103u...,"{'A': ' Portfolio A', 'B': ' Portfolio B', 'C'...",B,Portfolio B is inefficient (falls below the ef...,Knowledge reasoning QA,text+image,Easy,"Markowitz efficient frontier, portfolio theory",Foundation of Risk Management,foundation of risk management1,"[-0.021951964125037193, -0.006178832147270441,...","[0.00967503897845745, -0.029878409579396248, 0..."
96,2270,107,NaN,NaN,Consider the expected returns and standard dev...,images/Foundationofriskmanagement1_images/107u...,"{'A': ' Portfolio 1', 'B': ' Portfolio 2', 'C'...",A,Portfolio 1 is not efficient beca

In [17]:
data['QuestionTextEmbeding'][0]

[0.014189721085131168,
 0.02863846719264984,
 0.06598665565252304,
 0.0733688697218895,
 0.019572587683796883,
 0.01860124245285988,
 0.021418141201138496,
 0.01606765389442444,
 0.006358257494866848,
 0.022761834785342216,
 0.016197165474295616,
 -0.04944142326712608,
 -0.007406500168144703,
 0.013040296733379364,
 0.005289779044687748,
 -0.006321832071989775,
 0.055366624146699905,
 -0.013971168547868729,
 -0.06825312227010727,
 0.009915806353092194,
 0.005273589864373207,
 0.022421862930059433,
 0.0031163962557911873,
 0.0547514371573925,
 -0.03074304573237896,
 -0.013598820194602013,
 0.020090637728571892,
 -0.017516575753688812,
 -0.015873383730649948,
 0.019831612706184387,
 0.05941389128565788,
 -0.037008218467235565,
 -0.04675403609871864,
 -0.037364374846220016,
 -0.03354375436902046,
 0.012408923357725143,
 -0.02216283790767193,
 0.002381817204877734,
 0.02507687173783779,
 -0.0018243059748783708,
 0.028023282065987587,
 0.002215879037976265,
 -0.027683312073349953,
 -0.02230

In [19]:
import numpy as np


ValueError: zero-dimensional arrays cannot be concatenated

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

# 对查询文本生成embedding
query_text_embedding = openai.Embedding.create(
    input="Consider the expected returns and standard deviations for the following portfolios:",
    model="text-embedding-ada-002"
)['data'][0]['embedding']

# 计算与数据库中文本embedding的相似度
text_similarity_scores = cosine_similarity([query_text_embedding], database_text_embeddings)

# 获取Top N个相似结果
top_n_indices = np.argsort(text_similarity_scores[0])[-N:]  # 返回相似度最高的N个
